In [0]:
# Fetch parameters from Azure Data Factory
schemaName=dbutils.widgets.get("schemaName")
tableName=dbutils.widgets.get("tableName")
filePath=dbutils.widgets.get("filePath")

# Fetch schema data from landing zone
jsonSchema = spark.read.parquet(f"/Volumes/mycatalog/volumes/mnt_landing/\
{schemaName}/{filePath}/{tableName}.parquet").schema.json()
ddl = spark.sparkContext._jvm.org.apache.spark.sql.types.DataType \
  .fromJson(jsonSchema).toDDL()

# Migrate parquet data to delta files
(spark.readStream
  .format("cloudFiles")
  .option("cloudFiles.format", "parquet")
  .option("cloudFiles.includeExistingFiles", "true")
  .option("cloudFiles.backfillInterval", "1 day")
  .option("cloudFiles.schemaLocation", "/Volumes/mycatalog/volumes/mnt_landing/\
  adventureworks/_checkpoint/address_autoload/")
  .schema(ddl)
  .load(f"/Volumes/mycatalog/volumes/mnt_landing/\
  {schemaName}/{filePath}/{tableName}.parquet")
  .writeStream
  .format("delta")
  .option("checkpointLocation", "/Volumes/mycatalog/volumes/mnt_landing/\
  adventureworks/_checkpoint/address_autoload/")
  .trigger(availableNow=True)
  .toTable("bronze_adventureworks.address")
)